In [109]:
import seaborn as sns
import pandas as pd
import numpy as np
import functools
import operator
import collections

In [110]:
# Permanently changes the pandas settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Abkürzungen

* LS: Lohnsteuer
* GKV: Gesetzliche Krankenversicherung
* PKV: Private Krankenversicherung

## Parameter

In [111]:
# Termintechnische Daten
JAHR = 2024
EINTRITTSALTER = 23
ENDALTER = 100

# Personenbezigene Daten
LOHNSTEUER = 0.42
SOLIDARITAETSZUSCHLAG = 0
STEUERLAST = LOHNSTEUER + SOLIDARITAETSZUSCHLAG
NETTOEINKOMMEN = 3300 # p.M.
EINKOMMENSENTWICKLUNG = 0.03 # p.a.

# Rente
RENTENEINTRITTSALTER = 70
RENTENANSPRUCH = 2500 # im Startjahr p.M.
RENTENENTWICKLUNG = 0.02 # p.a.
RENTENZUSCHUSS_PKV = 0.081
RENTENZUSCHUSS_PKV_MAX_ANTEIL = 0.5 # maximaler Anteil, der von der Rentenversicherung bezuschusst wird. 2024 max. 50%

# GKV (2024)
# https://www.bundesgesundheitsministerium.de/beitraege
GKV_BEITRAGSBEMESSUNGSGRENZE = 5175 # p.M.
GKV_BEITRAGSBEMESSUNGSGRENZE_STEIGERUNG = 0.02 # p.a.
GKV_BEITRAG = 0.146 # Anteil vom Bruttogehalt
GKV_ZUSATZBEITRAG = 0.009 # Anteil vom Bruttogehalt
GKV_ANTEIL_ARBEITGEBER = 0.5
GKV_ANTEIL_RENTENVERSICHERUNG = 0.5
GKV_GESAMTBEITRAG = GKV_BEITRAG + GKV_ZUSATZBEITRAG
GKV_GESAMTBEITRAG_INKL_AG = GKV_GESAMTBEITRAG * (1 - GKV_ANTEIL_ARBEITGEBER)
GKV_GESAMTBEITRAG_INKL_RV = GKV_GESAMTBEITRAG * (1 - GKV_ANTEIL_RENTENVERSICHERUNG)

# Pflegeversicherung in GKV
# https://www.tk.de/firmenkunden/versicherung/beitraege-faq/pflegereform-2023/wie-hoch-ist-pv-beitrag-ab-01072023-2149454?tkcm=ab
GKV_PV_BEITRAG = 0.034 # Anteil vom Bruttogehalt
GKV_PV_KINDERLOSENZUSCHLAG = 0.006 # Anteil vom Bruttogehalt / kein AG Zuschuss
GKV_PV_ANTEIL_ARBEITGEBER = 0.5
GKV_PV_KINDER_ABSCHLAG_BIS = 24 # "unter 25 Jahre"
GKV_PV_KINDER_ABSCHLAG_STAFFELUNG = {
    2: 0.0025, 
    3: 0.005, 
    4: 0.0075, 
    5: 0.01,
    6: 0.01,
    7: 0.01,
    8: 0.01,
    9: 0.01,
    10: 0.01,
}

# Zusatzversicherungen in GKV
GKV_ZV_BEITRAG = 0
GKV_ZV_BEITRAGSSTEIGERUNG = 0.035 # p.a.

# PKV Tarif
PKV_VERSICHERUNG = 'Alte Oldenburger'
PKV_SELBSTBETEILIGUNG = {
    'max': 330,
    'anteil': 0.2,
}
PKV_TARIFKONDITIONEN = {
    'A 80/100': {
        'beitrag': 250.16,
        'steuerlich_ansetzbar': 0.97,
        'wegfall_renteneintritt': False,
    },
    'K 20': {
        'beitrag': 127.41,
        'steuerlich_ansetzbar': 0.6205,
        'wegfall_renteneintritt': False,
    },
    'K/S': {
        'beitrag': 131.33 - 127.41,
        'steuerlich_ansetzbar': 0,
        'wegfall_renteneintritt': False,
    },
    'Z 100/80': {
        'beitrag': 58.95,
        'steuerlich_ansetzbar': 0.6285,
        'wegfall_renteneintritt': False,
    },
    'AK (Variante AK-E)': {
        'beitrag': 0.95,
        'steuerlich_ansetzbar': 0,
        'wegfall_renteneintritt': False,
    },
    'KTV 6 140€': {
        'beitrag': 40.04,
        'steuerlich_ansetzbar': 0,
        'wegfall_renteneintritt': True,
    },
}
PKV_BEITRAGSSTEIGERUNG = 0.035 # p.a.
PKV_ANTEIL_ARBEITGEBER = 0.5
PKV_BEITRAGSENTLASTUNGSPAKET = {
    'bezeichnung': 'PBE 2.0 350€',
    'beitrag': 141.40,
    'steuerlich_ansetzbar': -1, # wenn auf -1 gesetz wird der Anteil automatisch berechnet (durchschnittliche Absetzbarkeit)
}
PKV_BEITRAGSENTLASTUNGSPAKET_WIRKSAM_AB = 64 # Beitragssenkung ab 65
PKV_GESETZLICHE_BEITRAGSENTLASTUNG_EINZAHLUNG_BIS = 60 # Einzahlung bis x Jahre
PKV_GESETZLICHE_BEITRAGSENTLASTUNG_ANTEIL = 0.1
PKV_PV = {
    'bezeichnung': 'PVN',
    'beitrag': 50.47,
    'steuerlich_ansetzbar': 1,
}
PKV_PV_ANTEIL_ARBEITGEBER = 0.5

# Kinder
KINDER = [
    {
        'alter_bei_geburt': 30,
        'versicherungsdauer': 23, # in Jahren
    },
    {
        'alter_bei_geburt': 32,
        'versicherungsdauer': 23, # in Jahren
    },
]
PKV_KINDER_BEITRAG = 150 # in 2024 (Inflation wird im Rechner beachtet)
PKV_KINDER_BEITRAGSSTEIGERUNG = 0.035 # p.a.

# Kapitalmarkt und Wirtschaft
INFLATIONSRATE = 0.025 # p.a.
RENDITE_KAPITALMARKT = 0.06 # p.a.
KAPITALERTRAGSSTEUER = 0.25

## Durchschnittliche steuerliche Ansetzbarkeit

In [112]:
beitrag, _, _ = functools.reduce(operator.add, map(collections.Counter, PKV_TARIFKONDITIONEN.values())).values()
beitrag += PKV_PV['beitrag']
durchschnittliche_ansetzbarkeit = (PKV_PV['beitrag'] / beitrag) * PKV_PV['steuerlich_ansetzbar']
for konditionen in PKV_TARIFKONDITIONEN.values():
    durchschnittliche_ansetzbarkeit += (konditionen['beitrag'] / beitrag) * konditionen['steuerlich_ansetzbar']
if PKV_BEITRAGSENTLASTUNGSPAKET['steuerlich_ansetzbar'] == -1:
    PKV_BEITRAGSENTLASTUNGSPAKET['steuerlich_ansetzbar'] = durchschnittliche_ansetzbarkeit
durchschnittliche_ansetzbarkeit

0.7693799210377892

## Einkommensentwicklung

In [113]:
df = pd.DataFrame()

df['Jahr'] = np.arange(JAHR, JAHR - EINTRITTSALTER + ENDALTER + 1)
df['Alter'] = np.arange(EINTRITTSALTER, ENDALTER + 1)
df[f'Inflation seit {JAHR}'] = np.power(1 + INFLATIONSRATE, df['Jahr'] - JAHR)
df['Rentenbezug'] = df['Alter'] >= RENTENEINTRITTSALTER
df['Einkommen p.M.'] = NETTOEINKOMMEN * np.power(1 + EINKOMMENSENTWICKLUNG, df['Alter'] - EINTRITTSALTER) * (1 - df['Rentenbezug'])
df['Rente p.M.'] = RENTENANSPRUCH * np.power(1 + RENTENENTWICKLUNG, df['Alter'] - EINTRITTSALTER) * df['Rentenbezug']

df

,Jahr,Alter,Inflation seit 2024,Rentenbezug,Einkommen p.M.,Rente p.M.
0,2024,23,1.000000,False,3300.000000,0.000000
1,2025,24,1.025000,False,3399.000000,0.000000
2,2026,25,1.050625,False,3500.970000,0.000000
3,2027,26,1.076891,False,3605.999100,0.000000
4,2028,27,1.103813,False,3714.179073,0.000000
5,2029,28,1.131408,False,3825.604445,0.000000
6,2030,29,1.159693,False,3940.372579,0.000000
7,2031,30,1.188686,False,4058.583756,0.000000
8,2032,31,1.218403,False,4180.341269,0.000000
9,2033,32,1.248863,False,4305.751507,0.000000


## GKV Beiträge

In [114]:
kinder_sortiert = sorted(KINDER, key=lambda k: k['alter_bei_geburt'])
alter_wenn_erstes_kind = kinder_sortiert[0]['alter_bei_geburt'] if len(kinder_sortiert) > 0 else 1000

anzahl_kinder_pv_relevant = np.zeros(ENDALTER - EINTRITTSALTER + 1)
for kind in KINDER:
    start = kind['alter_bei_geburt'] - EINTRITTSALTER
    ende = start + GKV_PV_KINDER_ABSCHLAG_BIS
    for i in range(start, ende + 1):
        anzahl_kinder_pv_relevant[i] += 1


df['GKV: Kinderlos'] = df['Alter'] < alter_wenn_erstes_kind
df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre'] = anzahl_kinder_pv_relevant.astype(int)
df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre PV Abschlag'] = df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre'].map(GKV_PV_KINDER_ABSCHLAG_STAFFELUNG).fillna(0)

df['GKV: Beitragsbemessungsgrenze p.M.'] = GKV_BEITRAGSBEMESSUNGSGRENZE * np.power(1 + GKV_BEITRAGSBEMESSUNGSGRENZE_STEIGERUNG, df['Alter'] - EINTRITTSALTER)

df['GKV: Beitrag inkl. AG Anteil p.M.'] = df['GKV: Beitragsbemessungsgrenze p.M.'] * GKV_GESAMTBEITRAG_INKL_AG * (1 - df['Rentenbezug'])
df['GKV: PV Beitrag inkl. AG Anteil p.M.'] = df['GKV: Beitragsbemessungsgrenze p.M.'] * (GKV_PV_BEITRAG * (1 - GKV_PV_ANTEIL_ARBEITGEBER) + GKV_PV_KINDERLOSENZUSCHLAG * df['GKV: Kinderlos'] - df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre PV Abschlag']) * (1 - df['Rentenbezug'])
df['GKV: Gesamt Beitrag inkl. AG Anteil p.M.'] = df['GKV: Beitrag inkl. AG Anteil p.M.'] + df['GKV: PV Beitrag inkl. AG Anteil p.M.']

df['GKV: Beitrag Rente inkl. RV Anteil p.M.'] = np.minimum(df['Rente p.M.'], df['GKV: Beitragsbemessungsgrenze p.M.']) * GKV_GESAMTBEITRAG_INKL_RV * df['Rentenbezug']
df['GKV: PV Beitrag Rente p.M.'] = np.minimum(df['Rente p.M.'], df['GKV: Beitragsbemessungsgrenze p.M.']) * (GKV_PV_BEITRAG + GKV_PV_KINDERLOSENZUSCHLAG * df['GKV: Kinderlos'] - df[f'GKV: Kinder bis {GKV_PV_KINDER_ABSCHLAG_BIS} Jahre PV Abschlag']) * df['Rentenbezug']
df['GKV: Gesamt Beitrag Rente inkl. RV Anteil p.M.'] = df['GKV: Beitrag Rente inkl. RV Anteil p.M.'] + df['GKV: PV Beitrag Rente p.M.']

df['GKV: Beitrag inkl. Zuschüsse exkl. LS'] = np.where(df['Rentenbezug'], df['GKV: Gesamt Beitrag Rente inkl. RV Anteil p.M.'], df['GKV: Gesamt Beitrag inkl. AG Anteil p.M.'])
df['GKV: Steuererstattung'] = df['GKV: Beitrag inkl. Zuschüsse exkl. LS'] * STEUERLAST
df['GKV: Beitrag inkl. Zuschüsse inkl. LS'] = df['GKV: Beitrag inkl. Zuschüsse exkl. LS'] - df['GKV: Steuererstattung']

df

,Jahr,Alter,Inflation seit 2024,Rentenbezug,Einkommen p.M.,Rente p.M.,GKV: Kinderlos,GKV: Kinder bis 24 Jahre,GKV: Kinder bis 24 Jahre PV Abschlag,GKV: Beitragsbemessungsgrenze p.M.,GKV: Beitrag inkl. AG Anteil p.M.,GKV: PV Beitrag inkl. AG Anteil p.M.,GKV: Gesamt Beitrag inkl. AG Anteil p.M.,GKV: Beitrag Rente inkl. RV Anteil p.M.,GKV: PV Beitrag Rente p.M.,GKV: Gesamt Beitrag Rente inkl. RV Anteil p.M.,GKV: Beitrag inkl. Zuschüsse exkl. LS,GKV: Steuererstattung,GKV: Beitrag inkl. Zuschüsse inkl. LS
0,2024,23,1.000000,False,3300.000000,0.000000,True,0,0.0000,5175.000000,401.062500,119.025000,520.087500,0.000000,0.000000,0.000000,520.087500,218.436750,301.650750
1,2025,24,1.025000,False,3399.000000,0.000000,True,0,0.0000,5278.500000,409.083750,121.405500,530.489250,0.000000,0.000000,0.000000,530.489250,222.805485,307.683765
2,2026,25,1.050625,False,3500.970000,0.000000,True,0,0.0000,5384.070000,417.265425,123.833610,541.099035,0.000000,0.000000,0.000000,541.099035,227.261595,313.837440
3,2027,26,1.076891,False,3605.999100,0.000000,True,0,0.0000,5491.751400,425.610734,126.310282,551.921016,0.000000,0.000000,0.000000,551.921016,231.806827,320.114189
4,2028,27,1.103813,False,3714.179073,0.000000,True,0,0.0000,5601.586428,434.122948,128.836488,562.959436,0.000000,0.000000,0.000000,562.959436,236.442963,326.516473
5,2029,28,1.131408,False,3825.604445,0.000000,True,0,0.0000,5713.618157,442.805407,131.413218,574.218625,0.000000,0.000000,0.000000,574.218625,241.171822,333.046802
6,2030,29,1.159693,False,3940.372579,0.000000,True,0,0.0000,5827.890520,451.661515,134.041482,585.702997,0.000000,0.000000,0.000000,585.702997,245.995259,339.707738
7,2031,30,1.188686,False,4058.583756,0.000000,False,1,0.0000,5944.448330,460.694746,101.055622,561.750367,0.000000,0.000000,0.000000,561.750367,235.935154,325.815213
8,2032,31,1.218403,False,4180.341269,0.000000,False,1,0.0000,6063.337297,469.908640,103.076734,572.985375,0.000000,0.000000,0.000000,572.985375,240.653857,332.331517
9,2033,32,1.248863,False,4305.751507,0.000000,False,2,0.0025,6184.604043,479.306813,89.676759,568.983572,0.000000,0.000000,0.000000,568.983572,238.973100,330.010472


## PKV Beiträge

### Basis

In [115]:
df['PKV: gesetzlicher Zuschlag zur Beitragsentlastung'] = df['Alter'] <= PKV_GESETZLICHE_BEITRAGSENTLASTUNG_EINZAHLUNG_BIS

# Beiträge vor Lohnsteuerbetrachtung
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Beitrag {tarif} exkl. LS'] = konditionen['beitrag'] * np.power(1 +  PKV_BEITRAGSSTEIGERUNG, df['Alter'] - EINTRITTSALTER) * np.where((df['Alter'] >= RENTENEINTRITTSALTER) & konditionen['wegfall_renteneintritt'], 0, 1)
df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] = PKV_BEITRAGSENTLASTUNGSPAKET['beitrag'] * np.power(1 +  PKV_BEITRAGSSTEIGERUNG, df['Alter'] - EINTRITTSALTER)
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS'] = PKV_PV['beitrag'] * np.power(1 + PKV_BEITRAGSSTEIGERUNG, df['Alter'] - EINTRITTSALTER)
df['PKV: Beitrag Gesamt exkl. PV exkl. LS'] = df.loc[:, df.columns.str.startswith('PKV: Beitrag ') & df.columns.str.endswith(' exkl. LS') & ~df.columns.str.contains('Gesamt') & ~df.columns.str.contains(PKV_PV["bezeichnung"])].sum(axis=1)

# Zuschüsse AG
df['PKV: AG Zuschuss Gesamt exkl. PV'] = np.minimum(df['GKV: Beitragsbemessungsgrenze p.M.'] * GKV_GESAMTBEITRAG, df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * (1 - PKV_ANTEIL_ARBEITGEBER) * (1 - df['Rentenbezug'])
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: AG Zuschuss {tarif}'] = (df[f'PKV: Beitrag {tarif} exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: AG Zuschuss Gesamt exkl. PV'] * (1 - df['Rentenbezug'])
df[f'PKV: AG Zuschuss {PKV_PV["bezeichnung"]}'] = np.minimum(df['GKV: Beitragsbemessungsgrenze p.M.'] * GKV_PV_BEITRAG, df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS']) * (1 - PKV_PV_ANTEIL_ARBEITGEBER) * (1 - df['Rentenbezug'])
df[f'PKV: AG Zuschuss {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}'] = (df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] / df['PKV: Beitrag Gesamt exkl. PV exkl. LS']) * df['PKV: AG Zuschuss Gesamt exkl. PV'] * (1 - df['Rentenbezug'])

# Zuschüsse RV
df['PKV: RV Zuschuss steuerfrei'] = np.minimum(df['Rente p.M.'] * RENTENZUSCHUSS_PKV, df['PKV: Beitrag Gesamt exkl. PV exkl. LS'] * RENTENZUSCHUSS_PKV_MAX_ANTEIL)

# Beiträge inkl. AG-Zuschuss
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Beitrag {tarif} inkl. Zuschüsse exkl. LS'] = df[f'PKV: Beitrag {tarif} exkl. LS'] - df[f'PKV: AG Zuschuss {tarif}']
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse exkl. LS'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS'] - df[f'PKV: AG Zuschuss {PKV_PV["bezeichnung"]}']
df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse exkl. LS'] = df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} exkl. LS'] - df[f'PKV: AG Zuschuss {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}']

# Steuererstattung
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Steuererstattung {tarif}'] = df[f'PKV: Beitrag {tarif} inkl. Zuschüsse exkl. LS'] * konditionen['steuerlich_ansetzbar'] * STEUERLAST
df[f'PKV: Steuererstattung {PKV_PV["bezeichnung"]}'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse exkl. LS'] * PKV_PV['steuerlich_ansetzbar'] * STEUERLAST
df[f'PKV: Steuererstattung {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}'] = df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse exkl. LS'] * PKV_BEITRAGSENTLASTUNGSPAKET['steuerlich_ansetzbar'] * STEUERLAST

# Beiträge nach Steuern
for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Beitrag {tarif} inkl. Zuschüsse inkl. LS'] = df[f'PKV: Beitrag {tarif} inkl. Zuschüsse exkl. LS'] - df[f'PKV: Steuererstattung {tarif}']
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse inkl. LS'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. Zuschüsse exkl. LS'] - df[f'PKV: Steuererstattung {PKV_PV["bezeichnung"]}']
df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse inkl. LS'] = df[f'PKV: Beitrag {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]} inkl. Zuschüsse exkl. LS'] - df[f'PKV: Steuererstattung {PKV_BEITRAGSENTLASTUNGSPAKET["bezeichnung"]}']

##############################################################
# AG Zuschuss


for tarif, konditionen in PKV_TARIFKONDITIONEN.items():
    df[f'PKV: Steuererstattung {tarif}'] = df[f'PKV: Beitrag {tarif} exkl. LS'] * konditionen['steuerlich_ansetzbar'] * STEUERLAST
    df[f'PKV: Beitrag {tarif} inkl. LS'] = df[f'PKV: Beitrag {tarif} exkl. LS'] - df[f'PKV: Steuererstattung {tarif}']


df[f'PKV: Steuererstattung {PKV_PV["bezeichnung"]}'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS'] * PKV_PV['steuerlich_ansetzbar'] * STEUERLAST
df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} inkl. LS'] = df[f'PKV: Beitrag {PKV_PV["bezeichnung"]} exkl. LS'] - df[f'PKV: Steuererstattung {PKV_PV["bezeichnung"]}']

df['PKV: Steuererstattung Beitragsentlastung'] = df['PKV: Beitrag Beitragsentlastung exkl. LS'] * PKV_BEITRAGSENTLASTUNGSPAKET['steuerlich_ansetzbar'] * STEUERLAST
df['PKV: Beitrag Beitragsentlastung inkl. LS'] = df['PKV: Beitrag Beitragsentlastung exkl. LS'] - df['PKV: Steuererstattung Beitragsentlastung']

df['PKV: Steuererstattung Gesamt'] = df.loc[:, df.columns.str.startswith('PKV: Steuererstattung ') & ~df.columns.str.contains('Gesamt')].sum(axis=1)
df['PKV: Beitrag Gesamt inkl. LS'] = df.loc[:, df.columns.str.startswith('PKV: Beitrag ') & df.columns.str.endswith(' inkl. LS') & ~df.columns.str.contains('Gesamt')].sum(axis=1)

df.loc[:, ~df.columns.str.startswith('GKV: ')]

KeyError: 'PKV: Beitrag Beitragsentlastung exkl. LS'